In [2]:
import numpy as np
import pandas as pd

#read the data
df = pd.read_csv('grocExample.csv')
df

,apple,beer,milk,pear,rice,meat
0,1,1,0,0,1,1
1,1,1,0,0,1,0
2,1,1,0,0,0,0
3,1,0,0,1,0,0
4,0,1,1,0,1,1
5,0,1,1,0,1,0
6,0,1,1,0,0,0
7,0,0,1,1,0,0


In [3]:
from itertools import combinations
import numpy as np
import pandas as pd

def compute(df, min_support=0.5):
    X = df.values
    
    ary_col_idx = np.arange(X.shape[1])
    
    support = (np.sum(X, axis=0) / float(X.shape[0])) # 各自項目的support值(單一項)
    
    support_dict = {1: support[support >= min_support]} # 單一項的suppoet
    itemset_dict = {1: ary_col_idx[support >= min_support].reshape(-1, 1)} # column no.
    max_itemset = 1

    while max_itemset:
        next_max_itemset = max_itemset + 1
        combin = combinations(np.unique(itemset_dict[max_itemset].flatten()),
                              r=next_max_itemset)
        frequent_items = []
        frequent_items_support = []

        for c in combin:
            together = X[:, c].sum(axis=1) == len(c)
            support = together.sum() / float(X.shape[0])
            if support >= min_support:
                frequent_items.append(c)
                frequent_items_support.append(support)

        if frequent_items:
            itemset_dict[next_max_itemset] = np.array(frequent_items)
            support_dict[next_max_itemset] = np.array(frequent_items_support)
            max_itemset = next_max_itemset
        else:
            max_itemset = 0

    all_res = []
    for k in sorted(itemset_dict):
        support = pd.Series(support_dict[k])
        itemsets = pd.Series([i for i in itemset_dict[k]])

        res = pd.concat((support, itemsets), axis=1)
        all_res.append(res)

    res_df = pd.concat(all_res)
    res_df.columns = ['support', 'itemsets']
    mapping = {idx: item for idx, item in enumerate(df.columns)}
    res_df['itemsets'] = res_df['itemsets'].apply(lambda x: [mapping[i]
                                                      for i in x])
    res_df = res_df.reset_index(drop=True)

    return res_df

In [3]:
compute(df, min_support=0.2)

,support,itemsets
0,0.500,[apple]
1,0.750,[beer]
2,0.500,[milk]
3,0.250,[pear]
4,0.500,[rice]
5,0.250,[meat]
6,0.375,"[apple, beer]"
7,0.250,"[apple, rice]"
8,0.375,"[beer, milk]"
9,0.500,"[beer, rice]"
